# Fine Tuning Llama on Custom Dataset

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install -q transformers accelerate peft datasets
!pip install -U bitsandbytes trl

In [ ]:
import os
import torch
import shutil
import warnings
from trl import SFTTrainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
from huggingface_hub import HfApi, HfFolder, Repository, notebook_login
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
notebook_login()

In [ ]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
# Define BitsAndBytesConfig for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16, 
    bnb_4bit_use_double_quant=True,
)

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [ ]:
# Load model with quantization config
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config, 
    trust_remote_code=True
)

In [ ]:
dataset = load_dataset(r"gbharti/finance-alpaca", split="train")
print(dataset)

In [ ]:
def format_prompt(sample):
    return f"### Instruction:\n{sample['instruction']}\n\n### Response:\n{sample['output']}"

dataset = dataset.map(lambda x: {"text": format_prompt(x)}, remove_columns=dataset.column_names)

In [ ]:
# LoRA Configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=64,  
    lora_alpha=64, 
    lora_dropout=0.1,
    bias="none",
)

In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  

In [ ]:
import transformers
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,  
    eval_dataset=None,  
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,  
        gradient_accumulation_steps=4,  
        warmup_ratio=0.03,  
        max_steps=4000,  
        learning_rate=1e-4,  
        logging_steps=10,  
        output_dir="outputsLlama", 
        optim="adamw_bnb_8bit",  
        save_strategy="epoch",  
        fp16=True,  
        report_to="none", 
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [ ]:
print(torch.cuda.is_available())  
print(torch.cuda.device_count())  

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
torch.cuda.empty_cache() 

In [ ]:
model.config.use_cache = False
trainer.train()

In [ ]:
new_model = "Meta-Llama-3.1-8B-Finance-FineTune" 

In [ ]:
trainer.model.save_pretrained(new_model) 

In [ ]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
from peft import PeftModel
peft_model = PeftModel.from_pretrained(base_model, new_model)
merged_model = peft_model.merge_and_unload()

In [ ]:
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

### **Code by: Sagar Thapliyal**

In [ ]:

from datasets import Dataset
import pandas as pd

# Load your custom FEIN dataset
df = pd.read_csv("fein_dataset.csv")  # Update path if needed
dataset = Dataset.from_pandas(df)


In [ ]:

# Format the dataset with FEIN-specific prompt style
def format_prompt(sample):
    return f"""You are FEIN, an AI Financial Assistant trained to give tailored financial advice.

### User Query:
{sample['instruction']}

### FEIN's Advice:
{sample['output']}"""

dataset = dataset.map(lambda x: {"text": format_prompt(x)}, remove_columns=dataset.column_names)


In [ ]:

# Split the dataset into training and evaluation sets
dataset = dataset.train_test_split(test_size=0.05)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]


In [ ]:

# Update the trainer with evaluation and logging
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        max_steps=3000,
        learning_rate=2e-4,
        logging_steps=10,
        output_dir="fein_outputs",
        optim="adamw_bnb_8bit",
        save_strategy="epoch",
        evaluation_strategy="steps",
        eval_steps=250,
        fp16=True,
        report_to="none"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


In [ ]:

# Save and merge the LoRA model
output_dir = "fein-ai-llama3-8b-v1"
trainer.train()
trainer.model.save_pretrained(output_dir)


In [ ]:

# Inference example to test the model
from transformers import pipeline

pipe = pipeline("text-generation", model=output_dir, tokenizer=tokenizer)
query = "How should a 25-year-old plan for retirement if they earn ₹50,000 per month?"
response = pipe(query, max_new_tokens=256, do_sample=True)[0]["generated_text"]
print(response)
